In [1]:
import pandas as pd
import os
import re
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
import matplotlib.pyplot as plt
from pandas.plotting import table 
import dataframe_image as dfi

# Loading Best Features and the scores

In [5]:
#loading feature names from best models
feature_names = pd.read_csv('Data/words_as_predictors_performance/jatjak2.csv')
feature_names = feature_names[['group', 'target', 'feature_names']]


In [6]:
feature_names

,group,target,feature_names
0,1,Depressed_F1,"('helped', 'advantage', 'applause', 'awe', 'ba..."
1,1,Anxiety_F2,"('helped', 'advantage', 'applause', 'bicycle',..."
2,2,Depressed_F1,"('aggressive', 'angry', 'barbecue', 'bell', 'c..."
3,2,Anxiety_F2,"('aggressive', 'angry', 'barbecue', 'bell', 'c..."
4,3,Depressed_F1,"('activist', 'ambulance', 'avalanche', 'busine..."
5,3,Anxiety_F2,"('activist', 'ambulance', 'avalanche', 'bonfir..."
6,4,Depressed_F1,"('accordion', 'apology', 'apricot', 'empty', '..."
7,4,Anxiety_F2,"('accordion', 'apology', 'apricot', 'bay', 'be..."
8,5,Depressed_F1,"('damaged', 'airport', 'analogy', 'arm', 'attr..."
9,5,Anxiety_F2,"('damaged', 'airport', 'analogy', 'arm', 'attr..."


In [7]:
#loading the 8 datasets into a list

# assign path
path, dirs, files = next(os.walk("Data/words_as_predictors"))
file_count = len(files)
# create empty list
df_list = []
 
# append datasets to the list
for i in range(file_count):
    #reading the data
    temp_df = pd.read_csv("Data/words_as_predictors/"+files[i])
    #reorder category levels
    word_names = list(temp_df.iloc[:, 1:61].columns)
    target_names = list(temp_df.iloc[:, 65:67])
    temp_df = temp_df.apply(lambda x: x.astype('category') if x.name in word_names else x)
    temp_df = temp_df.apply(lambda x: x.astype('category') if x.name in target_names else x)
    temp_df = temp_df.transform(lambda x: x.cat.set_categories([-1, 1]) if x.name in word_names else x)
    temp_df = temp_df.transform(lambda x: x.cat.set_categories([0,1]) if x.name in target_names else x)
    temp_df = temp_df.drop(['Anxiety', 'Depressed'], axis=1)
    temp_df = temp_df.rename(columns={'Depressed_F1': 'Depression', 'Anxiety_F2': 'Anxiety'})

    #appending to the list
    df_list.append(temp_df)
     
# display datasets
for dataset in df_list:
    display(dataset)

,ID,helped,advantage,applause,awe,banjo,banker,belief,bicycle,blue,...,whole,xylophone,F1,F2,F1_transformed,F2_transformed,Depression,Anxiety,PHQ9,GAD7
0,R_1BOWltIDcyNGKoH,-1,1,1,1,1,1,1,1,1,...,-1,-1,0.887753,1.154815,0.995467,1.162114,1,1,17,15
1,R_2bVMkt1hhhln2jP,-1,-1,-1,-1,1,1,1,-1,-1,...,-1,1,-1.296660,-0.820741,-2.038769,-1.048750,0,0,0,4
2,R_3nCM4LsfCsIjOo7,-1,-1,-1,-1,1,-1,-1,1,-1,...,1,1,0.927729,-1.227558,1.017550,-1.838252,1,0,11,2
3,R_wRBZJ21QBhdo7p7,-1,-1,1,1,-1,1,1,-1,-1,...,1,1,0.135663,0.729673,0.341253,0.859894,0,1,11,11
4,R_1JVecwADD1hEEhd,1,1,1,1,1,1,1,1,1,...,1,-1,0.265664,-0.662713,0.493820,-0.713643,0,0,8,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
357,R_pDZ2ZuUBqQRdMC5,-1,-1,-1,-1,1,1,-1,1,1,...,-1,1,0.500659,-0.144696,0.700827,0.063665,1,0,10,5
358,R_2y25DrzlaPhcmtt,-1,-1,-1,1,1,1,-1,1,1,...,-1,1,-0.495961,2.167786,-0.422681,1.964812,0,1,6,8
359,R_27ecbNVYxed0jd9,-1,-1,-1,-1,1,-1,-1,-1,1,...,-1,1,0.029661,-0.151589,0.212413,0.057833,0,0,5,1
360,R_2fBwux8TV0UvpVo,1,1,1,1,-1,1,-1,1,1,...,-1,-1,1.316136,1.770656,1.306331,1.635912,1,1,18,16


,ID,aggressive,angry,barbecue,bell,book,business,cab,cabbage,cafeteria,...,young,zone,F1,F2,F1_transformed,F2_transformed,Depression,Anxiety,PHQ9,GAD7
0,R_1Dr6S3X8XV7aNWq,-1,-1,-1,1,-1,1,1,-1,1,...,1,1,-0.495368,-0.397361,-0.419046,-0.251716,0,0,5,4
1,R_2rwqMlB1yZpbsmB,-1,-1,-1,1,-1,-1,1,1,-1,...,-1,-1,0.502047,-0.616587,0.701891,-0.625182,1,0,7,3
2,R_xfpq3YIIpCmr0hX,1,1,-1,1,1,1,-1,1,1,...,-1,-1,0.807825,1.743641,0.947474,1.623361,1,1,19,20
3,R_129IACvxn4XnW9u,-1,-1,-1,-1,-1,1,1,1,1,...,-1,-1,0.507370,-0.879450,0.706153,-1.176946,1,0,7,2
4,R_2pJe7TuHyUVWLEj,1,1,1,-1,-1,-1,-1,-1,1,...,-1,-1,-0.777101,-0.862499,-0.806352,-1.135567,0,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
373,R_2wspAAuPbm29eNA,1,-1,1,1,-1,-1,-1,-1,-1,...,1,-1,0.128452,-0.949058,0.329815,-1.326107,0,0,5,3
374,R_3rHQ2bcH0LvJ75M,1,1,1,1,1,1,1,1,1,...,1,-1,-0.380127,-0.621544,-0.278420,-0.638382,0,0,2,0
375,R_1fdWU7E9pyDH5Zf,1,1,-1,1,1,-1,1,1,1,...,1,1,0.469524,-1.046316,0.664085,-1.523086,1,0,10,2
376,R_30q0onT1a0cxvAW,-1,1,1,1,-1,1,1,1,1,...,1,-1,0.079551,0.379870,0.276691,0.563650,0,0,7,6


,ID,activist,ambulance,avalanche,bonfire,broccoli,businessman,camel,camera,cherry,...,vice,worth,F1,F2,F1_transformed,F2_transformed,Depression,Anxiety,PHQ9,GAD7
0,R_1FbPQrke1paVybl,-1,-1,1,-1,-1,-1,-1,1,-1,...,-1,1,-0.217606,-0.801512,-0.082011,-1.025960,0,0,6,3
1,R_1LCkV0plBScTIxu,1,1,-1,-1,1,1,1,1,1,...,1,-1,-0.490839,-0.936651,-0.415417,-1.292786,0,0,4,3
2,R_2Tz76JQL2jHYPp6,1,1,-1,-1,-1,-1,-1,-1,1,...,-1,-1,-0.748475,0.882903,-0.767250,0.976536,0,1,2,7
3,R_XjQDjJtVx656vfz,1,1,-1,1,-1,1,1,1,-1,...,1,1,-0.229845,-0.536828,-0.091193,-0.455653,0,0,8,5
4,R_1qffAI4F7bYGKWY,-1,1,1,-1,-1,1,-1,-1,-1,...,1,-1,0.545842,2.144200,0.725495,1.936931,1,1,15,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
406,R_QbjlBacwbwyxUDT,1,1,1,1,1,1,1,1,1,...,1,1,2.217628,1.111713,1.926188,1.117527,1,1,25,14
407,R_wKJD8UrGX4uQ7w5,-1,-1,1,-1,1,-1,-1,-1,-1,...,1,1,0.267599,-0.876053,0.496640,-1.170322,0,0,5,1
408,R_vq6FulYrVwxzzJT,1,1,1,1,1,-1,-1,-1,1,...,-1,-1,0.647315,-1.113146,0.816176,-1.608023,1,0,9,4
409,R_3NJ9reKhujZJqso,1,1,-1,-1,1,-1,1,1,-1,...,1,-1,0.054139,-0.746294,0.239716,-0.900321,0,0,8,7


,ID,accordion,apology,apricot,bay,beer,cathedral,cheetah,college,corkscrew,...,worker,zoo,F1,F2,F1_transformed,F2_transformed,Depression,Anxiety,PHQ9,GAD7
0,R_r2tpTfFLvWCrG0N,1,-1,-1,-1,1,-1,-1,-1,-1,...,-1,1,-0.140687,-0.290827,0.016218,-0.079507,0,0,7,7
1,R_1jHvmK6ZxB1pI0v,1,-1,1,1,-1,1,-1,-1,-1,...,-1,-1,-0.747362,-0.572413,-0.760014,-0.532242,0,0,1,1
2,R_1r2ZiHHbIJG3ZBv,1,1,-1,-1,-1,-1,1,1,-1,...,-1,1,-0.412833,-0.291041,-0.321922,-0.080342,0,0,3,2
3,R_2VBHRj1t6GMLotd,1,-1,-1,1,-1,1,1,1,-1,...,1,-1,-0.350416,1.238361,-0.240572,1.235470,0,1,9,9
4,R_3KoKrYqRHmPPvAs,1,-1,1,1,1,1,1,1,1,...,-1,1,0.063914,-0.272590,0.245711,-0.070333,0,0,4,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
364,R_269bCsGmvYWQzuD,1,1,-1,1,1,1,1,1,1,...,-1,-1,-1.296544,0.642276,-2.032168,0.786367,0,1,2,4
365,R_2QoebyLn6C3tTg3,1,1,1,1,1,-1,1,-1,1,...,1,1,0.111635,-0.086162,0.309684,0.112930,0,0,8,4
366,R_1FqXAg76AZMtgPc,-1,1,-1,1,-1,-1,1,-1,-1,...,1,1,1.294572,0.276627,1.285152,0.474185,1,0,16,12
367,R_PIryUjJfza0wS2d,1,-1,1,-1,-1,-1,1,-1,1,...,1,1,0.236618,-0.376689,0.456576,-0.202216,0,0,9,5


,ID,damaged,airport,analogy,arm,attribute,audience,bus,car,carriage,...,whine,yellow,F1,F2,F1_transformed,F2_transformed,Depression,Anxiety,PHQ9,GAD7
0,R_DO7WBLeWTraBEkh,1,1,1,1,1,-1,1,-1,-1,...,-1,1,1.614291,1.168208,1.517802,1.171973,1,1,22,11
1,R_2QJmilO7N7JtBpM,1,1,-1,-1,-1,-1,1,1,1,...,1,-1,1.890717,0.480741,1.706457,0.651694,1,1,17,18
2,R_12okEGnKC0uBOVD,1,-1,-1,-1,-1,-1,-1,1,-1,...,1,-1,-0.571934,-0.336596,-0.515546,-0.157416,0,0,4,5
3,R_sYltNfFHlnks9VL,-1,-1,-1,-1,-1,-1,1,-1,1,...,1,1,0.041351,-0.025115,0.228603,0.167529,0,0,8,7
4,R_3s0nXgZ0kv9dSZU,1,1,1,1,1,1,1,1,-1,...,1,1,-0.877673,-0.295843,-1.035154,-0.085766,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
353,R_2COZfWakbxxUyKL,-1,-1,1,-1,1,-1,1,-1,1,...,-1,-1,-0.551887,0.487111,-0.488191,0.654782,0,1,3,4
354,R_1kYTLMEicRgyWfH,-1,1,-1,-1,-1,1,1,1,1,...,1,-1,-0.514841,0.465141,-0.451965,0.637363,0,0,4,4
355,R_SVQVqy3JREzdMtj,1,-1,-1,-1,-1,1,1,1,-1,...,1,-1,-0.498257,-0.327602,-0.428143,-0.128009,0,0,3,2
356,R_2v8qwwdKqV1xw0t,-1,1,-1,1,1,-1,-1,-1,1,...,-1,-1,-0.826352,-0.528407,-0.926192,-0.445986,0,0,1,0


,ID,alligator,army,bagpipe,banana,bar,bed,blueberry,boy,bribe,...,umbrella,wit,F1,F2,F1_transformed,F2_transformed,Depression,Anxiety,PHQ9,GAD7
0,R_2AEbBswtDMx6dmI,1,1,1,-1,1,-1,1,1,-1,...,-1,1,-0.386227,0.567055,-0.285343,0.721173,0,1,3,3
1,R_zVkcRzZ7Dg0So5X,1,1,-1,-1,-1,1,-1,-1,-1,...,1,-1,0.388227,0.121059,0.589181,0.332451,1,0,9,8
2,R_2tMwJtTOoKQyggX,1,1,1,1,-1,1,-1,-1,-1,...,1,-1,0.312453,-1.228666,0.539442,-1.847339,0,0,7,1
3,R_2zuMIprq0a2shga,1,-1,1,-1,1,-1,-1,-1,-1,...,-1,1,0.322072,-0.724260,0.549086,-0.853891,0,0,7,5
4,R_3nBKLeznUVme0h3,1,1,1,-1,-1,1,1,1,1,...,1,1,-0.185709,-0.703447,-0.046174,-0.809233,0,0,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390,R_1LS9wIU4ihO850B,1,-1,-1,-1,-1,1,-1,1,1,...,1,-1,-1.144900,0.287288,-1.632750,0.485382,0,0,2,5
391,R_233NG3QRA7Z1DXa,1,1,1,-1,1,-1,-1,-1,-1,...,-1,1,0.463222,-0.982002,0.660978,-1.395363,1,0,8,6
392,R_3Jm8wdSSX4qd8sW,1,1,1,1,-1,-1,-1,1,1,...,1,-1,1.268266,1.647043,1.260842,1.550167,1,1,21,21
393,R_1Lb8XHsntEVaPhr,-1,1,-1,-1,-1,-1,-1,-1,-1,...,-1,1,-0.452972,0.537261,-0.365154,0.700827,0,1,5,7


,ID,accident,advice,animosity,ant,apartment,artist,author,baseball,big,...,van,whale,F1,F2,F1_transformed,F2_transformed,Depression,Anxiety,PHQ9,GAD7
0,R_TbISNkHJ4IeaWwF,1,1,-1,1,1,1,1,1,1,...,1,1,-0.818020,0.352191,-0.912864,0.543294,0,0,3,7
1,R_1Ad63eU2Ca1KXLP,1,-1,-1,1,1,1,-1,-1,1,...,-1,1,-0.666303,-0.514891,-0.653237,-0.423135,0,0,1,0
2,R_1j7hOSdsSgrjGa7,1,1,-1,-1,-1,1,-1,1,-1,...,-1,1,-0.828422,-0.629935,-0.963886,-0.656329,0,0,2,3
3,R_1q4hHLZJoTYJcE4,1,1,-1,-1,-1,-1,-1,1,-1,...,1,-1,-0.209060,0.174581,-0.075337,0.375843,0,0,6,8
4,R_tFlteVSB8FKP42J,1,1,1,1,-1,-1,-1,1,1,...,1,1,0.528217,0.810712,0.719018,0.919190,1,1,14,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
361,R_XzVZ2LjbSbErdN7,1,-1,1,-1,-1,1,1,1,-1,...,-1,-1,0.609497,-1.068286,0.781843,-1.550167,1,0,7,0
362,R_1JD9gIWOY7Uu2oX,1,1,1,-1,1,1,1,1,-1,...,-1,1,-0.403544,-0.849784,-0.312739,-1.105955,0,0,3,1
363,R_9NqISJzQXhMjhtf,1,1,1,1,-1,-1,1,1,-1,...,1,1,-0.359814,1.490234,-0.254292,1.417726,0,1,10,16
364,R_2Qt8VZbXncWRo8m,1,1,-1,1,-1,1,1,1,-1,...,-1,1,1.157134,-1.634252,1.178610,-2.434065,1,0,9,1


,ID,actor,agreement,asparagus,axe,ball,battle,beach,bee,belch,...,woe,wonder,F1,F2,F1_transformed,F2_transformed,Depression,Anxiety,PHQ9,GAD7
0,R_1QseubXuFkKITgO,-1,-1,-1,1,1,1,-1,1,1,...,-1,-1,-0.379659,1.174219,-0.277555,1.180278,0,1,6,12
1,R_55OzDBBbMqXzJwl,1,-1,-1,-1,1,1,-1,1,1,...,1,1,-0.983714,-0.016718,-1.284203,0.179771,0,0,0,2
2,R_51g1JaoyDoCYAaB,1,-1,-1,1,1,1,-1,1,-1,...,-1,-1,-0.045097,0.097808,0.139337,0.310993,0,0,7,6
3,R_cHYMbmoxKDTLvs5,1,-1,-1,1,1,1,1,1,1,...,-1,1,-0.280772,-0.998051,-0.160785,-1.422285,0,0,6,5
4,R_2WO6CXbbg0sRsgl,1,-1,1,-1,1,1,-1,-1,1,...,-1,1,-0.344585,-0.455657,-0.232874,-0.339491,0,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
370,R_1FPbs2u13Zq0kJI,1,1,-1,-1,-1,1,1,1,1,...,-1,1,-0.134317,0.053385,0.021209,0.251716,0,0,5,3
371,R_1dbjo2qaQ3oESqX,1,-1,-1,1,1,1,-1,1,1,...,-1,-1,1.069060,-0.176303,1.120638,0.036185,1,0,12,7
372,R_1NsmA90QNN4eehi,1,-1,1,1,1,1,1,1,-1,...,-1,-1,-0.970387,0.515200,-1.259003,0.680768,0,1,3,4
373,R_3RjPDzZ0yehm3oy,1,-1,1,1,-1,-1,-1,-1,1,...,-1,1,-0.037314,-0.805872,0.153207,-1.031607,0,0,6,4


## Computing VIF for best features 

In [8]:
vif_list = []
targets = ['Depressed_F1', 'Anxiety_F2']


for target in targets:
    
    group_nr = 1
    i = 0
    for dataset in df_list:
        #renaming
        if target == 'Depression':
            target = 'Depressed_F1'
        elif target == 'Anxiety':
            target = 'Anxiety_F2' 
        
        group_nr = i+1
        print(group_nr, target)
        best_features = feature_names[(feature_names['group'] == group_nr)]
        best_features = best_features[(best_features['target'] == target)]
        best_features = best_features[['feature_names']]
        best_features = best_features.squeeze()
        best_features = re.sub('[\(\)\']', '', best_features)
        best_features = best_features.replace(' ', '')
        best_features = best_features.split(',')
        #print(type(best_features))
        #display(best_features)
        
        dataset = dataset[dataset.columns.intersection(best_features)]
        dataset['intercept'] = 1
        display(dataset)
        
        #renaming
        if target == 'Depressed_F1':
            target = 'Depression'
        elif target == 'Anxiety_F2':
            target = 'Anxiety'
        
        vif = pd.DataFrame()
        vif['variable'] = dataset.columns
        vif['group'] = group_nr
        vif['target'] = target
        vif['VIF'] = [variance_inflation_factor(dataset.values, i) for i in range(dataset.shape[1])]
        #vif = vif[vif['Variable']!='intercept']
        
        vif_list.append(vif)
        i +=1

1 Depressed_F1


C:\Users\ah140\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,helped,advantage,applause,awe,banjo,banker,circus,feared,hope,mushroom,piano,snub,thunder,truce,intercept
0,-1,1,1,1,1,1,1,1,-1,1,-1,1,1,-1,1
1,-1,-1,-1,-1,1,1,1,-1,-1,1,-1,-1,1,-1,1
2,-1,-1,-1,-1,1,-1,1,-1,-1,1,1,-1,1,1,1
3,-1,-1,1,1,-1,1,1,1,1,1,1,1,1,-1,1
4,1,1,1,1,1,1,1,-1,-1,1,1,1,1,-1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
357,-1,-1,-1,-1,1,1,-1,1,-1,1,1,1,-1,-1,1
358,-1,-1,-1,1,1,1,1,1,-1,-1,1,1,-1,-1,1
359,-1,-1,-1,-1,1,-1,1,1,-1,-1,-1,1,1,1,1
360,1,1,1,1,-1,1,1,1,1,1,-1,1,1,-1,1


2 Depressed_F1


C:\Users\ah140\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


...

C:\Users\ah140\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,accident,advice,animosity,ant,apartment,artist,author,baseball,big,bird,...,sympathy,table,tea,terrorist,ticket,trombone,trust,van,whale,intercept
0,1,1,-1,1,1,1,1,1,1,1,...,1,-1,-1,1,1,1,-1,1,1,1
1,1,-1,-1,1,1,1,-1,-1,1,1,...,-1,-1,1,1,1,1,-1,-1,1,1
2,1,1,-1,-1,-1,1,-1,1,-1,-1,...,1,-1,-1,1,1,-1,-1,-1,1,1
3,1,1,-1,-1,-1,-1,-1,1,-1,-1,...,-1,1,1,1,-1,-1,-1,1,-1,1
4,1,1,1,1,-1,-1,-1,1,1,1,...,1,1,-1,1,-1,1,-1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
361,1,-1,1,-1,-1,1,1,1,-1,1,...,-1,-1,-1,1,-1,1,-1,-1,-1,1
362,1,1,1,-1,1,1,1,1,-1,1,...,-1,-1,1,1,-1,-1,1,-1,1,1
363,1,1,1,1,-1,-1,1,1,-1,1,...,1,-1,-1,1,1,-1,1,1,1,1
364,1,1,-1,1,-1,1,1,1,-1,1,...,1,-1,1,1,-1,1,1,-1,1,1


8 Anxiety_F2


C:\Users\ah140\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,actor,agreement,asparagus,axe,ball,battle,belch,child,company,debate,...,fallacy,flood,gratitude,ivy,lightning,loan,penguin,rum,storm,intercept
0,-1,-1,-1,1,1,1,1,-1,-1,1,...,-1,1,-1,-1,-1,1,-1,1,-1,1
1,1,-1,-1,-1,1,1,1,-1,1,1,...,-1,1,1,1,1,-1,1,-1,-1,1
2,1,-1,-1,1,1,1,-1,-1,1,-1,...,1,-1,-1,1,1,1,-1,1,1,1
3,1,-1,-1,1,1,1,1,1,1,-1,...,1,-1,-1,1,1,1,1,1,1,1
4,1,-1,1,-1,1,1,1,-1,1,1,...,1,1,-1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
370,1,1,-1,-1,-1,1,1,1,-1,-1,...,1,1,-1,1,1,1,1,-1,-1,1
371,1,-1,-1,1,1,1,1,-1,-1,-1,...,1,-1,1,1,1,-1,1,-1,-1,1
372,1,-1,1,1,1,1,-1,1,1,-1,...,-1,-1,1,1,-1,-1,1,-1,-1,1
373,1,-1,1,1,-1,-1,1,-1,-1,-1,...,-1,1,-1,-1,1,1,-1,1,1,1


In [9]:
for vif in vif_list:
    display(vif)

,variable,group,target,VIF
0,helped,1,Depression,1.066522
1,advantage,1,Depression,1.059695
2,applause,1,Depression,1.106877
3,awe,1,Depression,1.098977
4,banjo,1,Depression,1.054049
5,banker,1,Depression,1.065794
6,circus,1,Depression,1.081845
7,feared,1,Depression,1.068171
8,hope,1,Depression,1.090823
9,mushroom,1,Depression,1.048068


,variable,group,target,VIF
0,aggressive,2,Depression,1.052056
1,angry,2,Depression,1.136161
2,barbecue,2,Depression,1.025860
3,bell,2,Depression,1.042407
4,church,2,Depression,1.025409
5,dead,2,Depression,1.087433
6,football,2,Depression,1.066900
7,guard,2,Depression,1.032392
8,jealousy,2,Depression,1.131026
9,intercept,2,Depression,1.580071


,variable,group,target,VIF
0,activist,3,Depression,1.020469
1,ambulance,3,Depression,1.073071
2,avalanche,3,Depression,1.052519
3,businessman,3,Depression,1.039029
4,friendly,3,Depression,1.091463
5,green,3,Depression,1.020578
6,grievance,3,Depression,1.033966
7,mayor,3,Depression,1.140345
8,policeman,3,Depression,1.091006
9,vacation,3,Depression,1.029379


,variable,group,target,VIF
0,accordion,4,Depression,1.013043
1,apology,4,Depression,1.027158
2,apricot,4,Depression,1.013679
3,empty,4,Depression,1.019396
4,matinee,4,Depression,1.028931
5,reality,4,Depression,1.013004
6,wanted,4,Depression,1.013351
7,zoo,4,Depression,1.023761
8,intercept,4,Depression,1.345302


,variable,group,target,VIF
0,damaged,5,Depression,1.048699
1,airport,5,Depression,1.062767
2,analogy,5,Depression,1.055394
3,arm,5,Depression,1.091252
4,attribute,5,Depression,1.029467
5,audience,5,Depression,1.064624
6,car,5,Depression,1.037158
7,computer,5,Depression,1.047809
8,eye,5,Depression,1.048679
9,foot,5,Depression,1.119853


...

,variable,group,target,VIF
0,accordion,4,Anxiety,1.220777
1,apology,4,Anxiety,1.091349
2,apricot,4,Anxiety,1.164465
3,bay,4,Anxiety,1.143083
4,beer,4,Anxiety,1.133090
5,cathedral,4,Anxiety,1.151927
6,cheetah,4,Anxiety,1.118443
7,college,4,Anxiety,1.103278
8,corkscrew,4,Anxiety,1.240294
9,dandelion,4,Anxiety,1.135781


,variable,group,target,VIF
0,damaged,5,Anxiety,1.116460
1,airport,5,Anxiety,1.092695
2,analogy,5,Anxiety,1.075281
3,arm,5,Anxiety,1.092604
4,attribute,5,Anxiety,1.020545
5,audience,5,Anxiety,1.073276
6,bus,5,Anxiety,1.085957
7,chestnut,5,Anxiety,1.077778
8,court,5,Anxiety,1.056232
9,egg,5,Anxiety,1.090146


,variable,group,target,VIF
0,alligator,6,Anxiety,1.410102
1,army,6,Anxiety,1.265758
2,bagpipe,6,Anxiety,1.324495
3,banana,6,Anxiety,1.284755
4,bar,6,Anxiety,1.219205
5,bed,6,Anxiety,1.297448
6,blueberry,6,Anxiety,1.316446
7,bribe,6,Anxiety,1.244821
8,butterfly,6,Anxiety,1.473869
9,carnival,6,Anxiety,1.240352


,variable,group,target,VIF
0,accident,7,Anxiety,1.320277
1,advice,7,Anxiety,1.218770
2,animosity,7,Anxiety,1.257212
3,ant,7,Anxiety,1.167011
4,apartment,7,Anxiety,1.173952
5,artist,7,Anxiety,1.202343
6,author,7,Anxiety,1.174960
7,baseball,7,Anxiety,1.226933
8,big,7,Anxiety,1.135130
9,bird,7,Anxiety,1.237801


,variable,group,target,VIF
0,actor,8,Anxiety,1.109023
1,agreement,8,Anxiety,1.075613
2,asparagus,8,Anxiety,1.105225
3,axe,8,Anxiety,1.116113
4,ball,8,Anxiety,1.107291
5,battle,8,Anxiety,1.106287
6,belch,8,Anxiety,1.113800
7,child,8,Anxiety,1.119096
8,company,8,Anxiety,1.072662
9,debate,8,Anxiety,1.122511


In [10]:
for vif in vif_list:
    print(vif[['VIF']].max())

VIF    1.690081
dtype: float64
VIF    1.580071
dtype: float64
VIF    1.863244
dtype: float64
VIF    1.345302
dtype: float64
VIF    1.420468
dtype: float64
VIF    2.442618
dtype: float64
VIF    2.047607
dtype: float64
VIF    1.437057
dtype: float64
VIF    2.08198
dtype: float64
VIF    2.192257
dtype: float64
VIF    2.533306
dtype: float64
VIF    1.881084
dtype: float64
VIF    1.658116
dtype: float64
VIF    3.133932
dtype: float64
VIF    3.193377
dtype: float64
VIF    1.829713
dtype: float64


In [11]:
#saving as images
for vif in vif_list:
    group_nr = str(vif.iloc[0,1])
    target = vif.iloc[0,2]
    name = group_nr + target
    filename = 'figures/words_as_predictors/words_as_predictors_vif/' + name + '.png'
    print(filename)
    vif = vif.drop(['group', 'target'], axis=1)
    dfi.export(vif, filename)

figures/words_as_predictors/words_as_predictors_vif/1Depression.svg
figures/words_as_predictors/words_as_predictors_vif/2Depression.svg
figures/words_as_predictors/words_as_predictors_vif/3Depression.svg
figures/words_as_predictors/words_as_predictors_vif/4Depression.svg
figures/words_as_predictors/words_as_predictors_vif/5Depression.svg
figures/words_as_predictors/words_as_predictors_vif/6Depression.svg
figures/words_as_predictors/words_as_predictors_vif/7Depression.svg
figures/words_as_predictors/words_as_predictors_vif/8Depression.svg
figures/words_as_predictors/words_as_predictors_vif/1Anxiety.svg
figures/words_as_predictors/words_as_predictors_vif/2Anxiety.svg
figures/words_as_predictors/words_as_predictors_vif/3Anxiety.svg
figures/words_as_predictors/words_as_predictors_vif/4Anxiety.svg
figures/words_as_predictors/words_as_predictors_vif/5Anxiety.svg
figures/words_as_predictors/words_as_predictors_vif/6Anxiety.svg
figures/words_as_predictors/words_as_predictors_vif/7Anxiety.svg
f